#coding:utf-8
'''
    FileName      ：processPrice.py
    Author        ：@zch0423
    Date          ：Jan 14, 2021
    Description   ：
    以机械设备行业为例，处理股价信息
    - 数据清洗，标准化
    - 提取季度数据，取3 6 9 12月末收盘价，导出为price处理后.xlsx
    - 选择上市超过15年的公司，即空值不超过20，导出为price15year.xlsx
    - 去除中间中断的公司（潍柴重机000880.SZ）导出为price59c.xlsx
    剩下59家公司
'''

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

## 预处理价格数据

In [3]:
filePath = "price前复权.xlsx"
raw = pd.read_excel(filePath)

In [4]:
raw.head()

,日期,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,*ST沈机000410.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,...,正帆科技688596.SH,皖仪科技688600.SH,奥普特688686.SH,创远仪器831961.OC,三友科技834475.OC,连城数控835368.OC,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
0,1992-05-29,1.1737,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1,1992-06-30,1.0777,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
2,1992-07-31,1.1067,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
3,1992-08-31,0.9818,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
4,1992-09-30,0.9110,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--


In [5]:
# 将时间设置为index
raw["日期"] = pd.to_datetime(raw["日期"])
raw.set_index("日期", inplace=True)
raw.head()

,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,*ST沈机000410.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,哈工智能000584.SZ,...,正帆科技688596.SH,皖仪科技688600.SH,奥普特688686.SH,创远仪器831961.OC,三友科技834475.OC,连城数控835368.OC,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
日期,,,,,,,,,,,,,,,,,,,,,
1992-05-29,1.1737,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-06-30,1.0777,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-07-31,1.1067,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-08-31,0.9818,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-09-30,0.9110,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--


In [6]:
# 删除ST
drop = [each for each in raw.columns if "ST" in each]
raw.drop(drop, axis=1, inplace=True)
raw.head()

,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,哈工智能000584.SZ,山推股份000680.SZ,...,正帆科技688596.SH,皖仪科技688600.SH,奥普特688686.SH,创远仪器831961.OC,三友科技834475.OC,连城数控835368.OC,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
日期,,,,,,,,,,,,,,,,,,,,,
1992-05-29,1.1737,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-06-30,1.0777,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-07-31,1.1067,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-08-31,0.9818,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-09-30,0.9110,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--


## 选出机械设备的股票季度收盘价
选择3、6、9、12月份四个月份的收盘价

In [7]:
# 从中筛选出机械的股票
machineROEFile = "机械roeData.xlsx"
roe = pd.read_excel(machineROEFile)
roe.head()

,code,name,2000-03,2000-06,2000-09,2000-12,2001-03,2001-06,2001-09,2001-12,...,2018-09,2018-12,2019-03,2019-06,2019-09,2019-12,2020-03,2020-06,2020-09,2020-12
0,000008.SZ,神州高铁,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.8794,2.7954,0.1604,0.7463,1.1356,3.9001,-1.4616,-2.5378,0.7020,NaN
1,000039.SZ,中集集团,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.5906,...,4.2918,3.0535,1.0623,0.7117,-0.1142,2.3391,-1.6447,1.1583,2.1542,NaN
2,000157.SZ,中联重科,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.7083,...,1.1780,1.8944,2.5903,4.1326,2.4141,2.3247,2.6003,7.0930,3.6814,NaN
3,000425.SZ,徐工机械,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.5403,1.8615,3.4380,3.9074,2.2892,1.8277,1.8240,4.2272,1.1084,NaN
4,000528.SZ,柳工,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.2561,0.7812,3.1375,3.6226,2.2035,1.3220,1.6257,4.7731,2.8909,NaN


In [8]:
# name+code获得一列
df = pd.DataFrame()
df["name"] = roe["name"]+roe["code"]
values = df["name"].values  # array
print(len(values))

398


In [9]:
data = raw[values]

In [10]:
data.head()

,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,哈工智能000584.SZ,山推股份000680.SZ,...,正帆科技688596.SH,皖仪科技688600.SH,奥普特688686.SH,创远仪器831961.OC,三友科技834475.OC,连城数控835368.OC,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
日期,,,,,,,,,,,,,,,,,,,,,
1992-05-29,1.1737,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-06-30,1.0777,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-07-31,1.1067,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-08-31,0.9818,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
1992-09-30,0.9110,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--


In [11]:
data.replace("--",np.nan,inplace=True)

In [12]:
data.head()

,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,哈工智能000584.SZ,山推股份000680.SZ,...,正帆科技688596.SH,皖仪科技688600.SH,奥普特688686.SH,创远仪器831961.OC,三友科技834475.OC,连城数控835368.OC,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
日期,,,,,,,,,,,,,,,,,,,,,
1992-05-29,1.1737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-06-30,1.0777,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-07-31,1.1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-08-31,0.9818,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1992-09-30,0.9110,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# 提取2000-2020季度数据 3 6 9 10
dates = []
for year in range(2000, 2021):
    for month in ["03", "06", "09", "12"]:
        dates.append(str(year)+"-"+month)

In [36]:
final = pd.concat([data[d] for d in dates])

In [37]:
final

,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,哈工智能000584.SZ,山推股份000680.SZ,...,正帆科技688596.SH,皖仪科技688600.SH,奥普特688686.SH,创远仪器831961.OC,三友科技834475.OC,连城数控835368.OC,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
日期,,,,,,,,,,,,,,,,,,,,,
2000-03-31,6.8169,1.7141,NaN,1.0030,0.9613,2.1426,6.1304,6.3989,2.8847,1.3026,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0914,0.1255,0.1274,0.0227
2000-06-30,6.6332,2.7344,NaN,1.3443,1.1987,2.6916,5.3171,6.0259,3.9702,3.0431,...,NaN,NaN,NaN,NaN,NaN,NaN,0.1208,0.1724,0.1532,0.0301
2000-09-29,5.0766,2.2704,NaN,1.3480,1.3141,2.4075,4.5199,6.4132,4.7754,3.1549,...,NaN,NaN,NaN,NaN,NaN,NaN,0.1364,0.2026,0.1497,0.0319
2000-12-29,4.7376,2.5609,0.3387,1.6333,1.3860,3.0181,4.9933,6.2937,4.0056,2.6120,...,NaN,NaN,NaN,NaN,NaN,NaN,0.1968,0.3448,0.2146,0.0504
2001-03-30,2.4107,2.8469,0.3196,1.6032,1.3442,2.7422,4.9407,6.2602,4.0881,2.3382,...,NaN,NaN,NaN,NaN,NaN,NaN,0.3220,0.6298,0.3871,0.0828
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31,3.6178,9.6612,6.5049,5.4179,6.7808,3.8986,6.6800,4.5900,6.2581,3.6500,...,NaN,NaN,NaN,10.8875,5.7786,30.0,0.4359,0.4580,1.4183,0.3208
2020-03-31,3.1294,7.5853,5.5408,5.0019,6.1164,3.7002,7.3400,4.2700,5.3998,3.1800,...,NaN,NaN,NaN,18.8510,7.8369,42.1,0.3769,0.4070,1.0520,0.2809
2020-06-30,2.9400,7.2300,6.2614,5.8537,6.3100,3.3134,9.8200,4.4500,5.5300,3.4300,...,NaN,NaN,NaN,24.1500,10.3977,44.3,0.3237,0.3400,0.9707,0.2080


In [47]:
final.to_excel("price处理后.xlsx")

Int64Index([ 5,  6,  7,  8,  9, 10, 11, 12,  1,  2,
            ...
             4,  5,  5,  6,  7,  8,  9, 10, 11, 12],
           dtype='int64', name='日期', length=346)

In [57]:
# 保留上市超过15年的公司 总共80季度 剩60家
price15 = final.loc[:, final.isnull().sum(axis=0)<=20]
price15.head()

,神州高铁000008.SZ,中集集团000039.SZ,中联重科000157.SZ,徐工机械000425.SZ,柳工000528.SZ,冰山冷热000530.SZ,创元科技000551.SZ,苏常柴A000570.SZ,哈工智能000584.SZ,山推股份000680.SZ,...,上海机电600835.SH,上柴股份600841.SH,上工申贝600843.SH,广日股份600894.SH,建设机械600984.SH,贵绳股份600992.SH,上柴B股900920.SH,上工B股900924.SH,机电B股900925.SH,振华B股900947.SH
日期,,,,,,,,,,,,,,,,,,,,,
2000-03-31,6.8169,1.7141,NaN,1.0030,0.9613,2.1426,6.1304,6.3989,2.8847,1.3026,...,4.9487,5.9088,6.8043,2.3801,NaN,NaN,0.0914,0.1255,0.1274,0.0227
2000-06-30,6.6332,2.7344,NaN,1.3443,1.1987,2.6916,5.3171,6.0259,3.9702,3.0431,...,4.9106,5.8948,6.9603,2.6776,NaN,NaN,0.1208,0.1724,0.1532,0.0301
2000-09-29,5.0766,2.2704,NaN,1.3480,1.3141,2.4075,4.5199,6.4132,4.7754,3.1549,...,4.7693,5.7915,7.1729,3.8375,NaN,NaN,0.1364,0.2026,0.1497,0.0319
2000-12-29,4.7376,2.5609,0.3387,1.6333,1.3860,3.0181,4.9933,6.2937,4.0056,2.6120,...,5.8953,6.8016,8.3448,4.2582,NaN,NaN,0.1968,0.3448,0.2146,0.0504
2001-03-30,2.4107,2.8469,0.3196,1.6032,1.3442,2.7422,4.9407,6.2602,4.0881,2.3382,...,5.8482,6.6221,8.4960,4.8635,NaN,NaN,0.3220,0.6298,0.3871,0.0828


In [58]:
price15.to_excel("price15year.xlsx")

In [60]:
# 去掉 潍柴重机000880.SZ中间缺失
price15.drop("潍柴重机000880.SZ", axis=1, inplace=True)

In [61]:
price15.to_excel("price59c.xlsx")  # 剩下59个